In [1]:
import pandas as pd
import numpy as np 

import statistics
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 25)

In [3]:
countries_data = pd.read_csv("Data/all-weeks-countries.csv", encoding = 'cp1252')
countries_data.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2023-03-05,Films,1,We Have a Ghost,NaN,2
1,Argentina,AR,2023-03-05,Films,2,Bad Boys for Life,NaN,2
2,Argentina,AR,2023-03-05,Films,3,The Condemned,NaN,1
3,Argentina,AR,2023-03-05,Films,4,Love at First Kiss,NaN,1
4,Argentina,AR,2023-03-05,Films,5,Tonight You're Sleeping with Me,NaN,1


In [4]:
# countries_data.country_name.nunique()
# countries_data.week.max()
countries_data.category.value_counts()

Films    82190
TV       82190
Name: category, dtype: int64

In [5]:
global_data = pd.read_csv("Data/all-weeks-global.csv", encoding = 'cp1252')
global_data.head()

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,cumulative_weeks_in_top_10
0,2023-03-05,Films (English),1,We Have a Ghost,NaN,42620000,2
1,2023-03-05,Films (English),2,The Strays,NaN,14570000,2
2,2023-03-05,Films (English),3,The Condemned,NaN,10230000,1
3,2023-03-05,Films (English),4,R.I.P.D.,NaN,8330000,1
4,2023-03-05,Films (English),5,Bad Boys for Life,NaN,8330000,2


In [6]:
global_data.week.min()

'2021-07-04'

In [7]:
popular_data = pd.read_csv("Data/most-popular.csv", encoding = 'cp1252')
popular_data.head()

,category,rank,show_title,season_title,hours_viewed_first_28_days
0,Films (English),1,Red Notice,NaN,364020000
1,Films (English),2,Don't Look Up,NaN,359790000
2,Films (English),3,Bird Box,NaN,282020000
3,Films (English),4,Glass Onion: A Knives Out Mystery,NaN,279740000
4,Films (English),5,The Gray Man,NaN,253870000


In [8]:
popular_data.category.value_counts()

Films (English)        10
Films (Non-English)    10
TV (English)           10
TV (Non-English)       10
Name: category, dtype: int64

In [9]:
shows_data = pd.read_csv("Data/netflix_shows.csv", encoding = 'cp1252')
shows_data.head()

,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type
0,Breaking Bad,2008.0,18+,9.5,96%,1,0,0,0,1.0
1,Stranger Things,2016.0,16+,8.8,93%,1,0,0,0,1.0
2,Money Heist,2017.0,18+,8.4,91%,1,0,0,0,1.0
3,Sherlock,2010.0,16+,9.1,78%,1,0,0,0,1.0
4,Better Call Saul,2015.0,18+,8.7,97%,1,0,0,0,1.0


In [10]:
shows_data.isnull().sum()

Title                 3
Year                  1
Age                2446
IMDb               1161
Rotten Tomatoes    4599
Netflix               0
Hulu                  0
Prime Video           0
Disney+               0
type                  1
dtype: int64

In [11]:
shows_data.columns = shows_data.columns.str.replace(" ","_") 

In [12]:
shows_data.shape

(5611, 10)

In [13]:
shows_data = shows_data.dropna(subset=['Year'])

In [14]:
# shows_data.Year.value_counts()

In [15]:
# Assuming shows_data is your DataFrame with the 'Year' column
shows_data['Year'] = shows_data['Year'].replace([np.inf, -np.inf], np.nan)  # Replace inf values with NaN
shows_data['Year'] = shows_data['Year'].fillna(0)  # Replace NaN values with 0
shows_data['Year'] = shows_data['Year'].astype('int')  # Convert to integer

In [16]:
shows_data.query('Year == 0')

,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney+,type


In [17]:
shows_data.query('Year == 0')

,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney+,type


In [18]:
shows_data.sample(2)

,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney+,type
3552,Food Fetish,2011,NaN,NaN,NaN,0,1,0,0,1.0
3546,WIRED's Autocomplete Interviews,2016,NaN,NaN,NaN,0,1,1,0,1.0


## Statistical Analysis

### 1. Is there a significant difference in IMDb ratings between shows available on Netflix, Hulu, Prime Video, and Disney+?

In [19]:
shows_data.Netflix.value_counts()

0    3679
1    1931
Name: Netflix, dtype: int64

In [20]:
shows_data.Hulu.value_counts()

0    3856
1    1754
Name: Hulu, dtype: int64

In [21]:
shows_data['Prime_Video'].value_counts()

0    3467
1    2143
Name: Prime_Video, dtype: int64

In [22]:
shows_data['Disney+'].value_counts()

0    5430
1     180
Name: Disney+, dtype: int64

In [23]:
## assume significance level = 0.05

alpha = 0.05

## extract the data, make sure no null value is selected and sampling it for better random results

netflix = shows_data[shows_data.Netflix == 1]['IMDb'].dropna()
hulu = shows_data[shows_data.Hulu == 1]['IMDb'].dropna()
prime_video = shows_data[shows_data.Prime_Video == 1]['IMDb'].dropna()
disney = shows_data[shows_data['Disney+'] == 1]['IMDb'].dropna()

In [24]:
## applying anova test

f_statistics, p_value = stats.f_oneway(netflix, hulu, prime_video, disney)

## results

print(f"P-value for the ANOVA test is {p_value} and F-Statistics value is {f_statistics}")


## conclusion
if p_value > alpha:
    print(f"As P-value, {p_value} is greater than significance level, {alpha}, so we fail in Rejecting NULL Hypothesis, conclude that there is no significant difference.")
else:
    print(f"As P-value,{p_value} is smaller than significance level, {alpha}, so we will be Rejecting NULL Hypothesis conclude that there is a significant difference in IMDb ratings among the streaming platforms.")


P-value for the ANOVA test is 0.0016406599840925664 and F-Statistics value is 5.079692433450253
As P-value,0.0016406599840925664 is smaller than significance level, 0.05, so we will be Rejecting NULL Hypothesis conclude that there is a significant difference in IMDb ratings among the streaming platforms.


In [25]:
## manually

netflix.mean(), hulu.mean(), prime_video.mean(), disney.mean()

(7.1632799558255105, 7.0611074489123276, 7.1798010711553175, 6.923668639053254)

### 2. Does the age rating (e.g., 16+, 18+) of TV shows have an impact on their IMDb ratings?


In [26]:
shows_data.columns

Index(['Title', 'Year', 'Age', 'IMDb', 'Rotten_Tomatoes', 'Netflix', 'Hulu',
       'Prime_Video', 'Disney+', 'type'],
      dtype='object')

In [27]:
shows_data.Age.value_counts()

16+    1018
7+      848
18+     750
all     545
13+       4
Name: Age, dtype: int64

In [28]:
# shows_data.query("Age == ['3.1', '7.2', '7.8']")  #data cleaning

In [29]:
## filtering population and taking only 50% of population

age_7 = shows_data[shows_data['Age'] == '7+']['IMDb'].dropna()
age_13 = shows_data[shows_data['Age'] == '13+']['IMDb'].dropna()
age_16 = shows_data[shows_data['Age'] == '16+']['IMDb'].dropna()
age_18 = shows_data[shows_data['Age'] == '18+']['IMDb'].dropna()
age_all = shows_data[shows_data['Age'] == 'all']['IMDb'].dropna()

In [30]:
age_7.shape, age_13.shape, age_16.shape, age_18.shape, age_all.shape

((838,), (4,), (1013,), (749,), (532,))

In [31]:
age_7.mean(), age_13.mean(), age_16.mean(), age_18.mean(), age_all.mean()

(7.062171837708831,
 6.0,
 7.2755182625863775,
 7.398130841121495,
 6.874624060150376)

In [32]:
age_13

1116    6.6
1123    6.4
1514    5.7
4024    5.3
Name: IMDb, dtype: float64

In [33]:
datapoints = [age_7, age_13, age_16, age_18, age_all]

In [34]:
## plotting distplot to visulaize the dats

def displot_func(data):
    plt.figure(figsize = (8,5))
    sns.displot(data)
    plt.axvline(x = data.mean(), color = 'red', linestyle = 'dotted', linewidth = 2)
    # plt.title("Distplot of the", data)
    plt.show()

**we choose a t-test when comparing the means of two groups, while ANOVA is used when comparing the means of more than two groups.**

In [35]:
## using ANOVA test due to different categorical values in a column

## using 50% samples from each category

## significance level = 0.50
alpha = 0.50

f_statistics, p_value = stats.f_oneway(age_7, age_16, age_18, age_all)

print(f"The p-value for the test is {p_value} and f_statistics value is {f_statistics}")

print("\nConclusion : \n")

if p_value > alpha:
    print(f"P-value for the following test is {p_value} and that's greater than significance value, so we cannot REJECT the NULL Hypothesis, means there is no age effects in IMDb ratings")
else:
    print("We REJECT the NULL Hypothesis and conclude that there is a signifance effects of age ratings on IMDb ratings")

The p-value for the test is 4.056435723903755e-19 and f_statistics value is 30.00355682223527

Conclusion : 

We REJECT the NULL Hypothesis and conclude that there is a signifance effects of age ratings on IMDb ratings


### 3. Are there significant differences in IMDb ratings among TV shows in different categories (e.g., Films, Series)?


In [36]:
shows_data.columns

Index(['Title', 'Year', 'Age', 'IMDb', 'Rotten_Tomatoes', 'Netflix', 'Hulu',
       'Prime_Video', 'Disney+', 'type'],
      dtype='object')

In [37]:
shows_data.type.value_counts()

1.0    5610
Name: type, dtype: int64

In [38]:
popular_data.category.value_counts()

Films (English)        10
Films (Non-English)    10
TV (English)           10
TV (Non-English)       10
Name: category, dtype: int64

In [39]:
countries_data.category.value_counts()

Films    82190
TV       82190
Name: category, dtype: int64

In [40]:
shows_data.shape

(5610, 10)

In [41]:
global_data.columns, global_data.shape

(Index(['week', 'category', 'weekly_rank', 'show_title', 'season_title',
        'weekly_hours_viewed', 'cumulative_weeks_in_top_10'],
       dtype='object'),
 (3520, 7))

In [42]:
global_data.category.value_counts()

Films (English)        880
Films (Non-English)    880
TV (English)           880
TV (Non-English)       880
Name: category, dtype: int64

#### Merging `Global Data` with 'Shows Data` on title column and it's an inner join

In [43]:
glob_show_data = pd.merge(global_data, shows_data, left_on = 'show_title' , right_on = 'Title', how = 'inner')

In [44]:
glob_show_data.Title.nunique()

97

In [45]:
glob_show_data.shape

(588, 17)

In [46]:
glob_show_data.head()

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,cumulative_weeks_in_top_10,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney+,type
0,2023-03-05,TV (English),1,Outer Banks,Outer Banks: Season 3,99000000,2,Outer Banks,2020,18+,7.6,72%,1,0,0,0,1.0
1,2023-03-05,TV (English),4,Outer Banks,Outer Banks: Season 1,34050000,7,Outer Banks,2020,18+,7.6,72%,1,0,0,0,1.0
2,2023-03-05,TV (English),6,Outer Banks,Outer Banks: Season 2,27750000,8,Outer Banks,2020,18+,7.6,72%,1,0,0,0,1.0
3,2023-02-26,TV (English),1,Outer Banks,Outer Banks: Season 3,154970000,1,Outer Banks,2020,18+,7.6,72%,1,0,0,0,1.0
4,2023-02-26,TV (English),4,Outer Banks,Outer Banks: Season 1,35100000,6,Outer Banks,2020,18+,7.6,72%,1,0,0,0,1.0


In [47]:
glob_show_data.category.value_counts()

TV (English)           410
TV (Non-English)       145
Films (English)         26
Films (Non-English)      7
Name: category, dtype: int64

In [48]:
## filtering data and considering not null values only

tv_eng = glob_show_data[glob_show_data.category == 'TV (English)']['IMDb'].dropna()
tv_noneng = glob_show_data[glob_show_data.category == 'TV (Non-English)']['IMDb'].dropna()
films_eng = glob_show_data[glob_show_data.category == 'Films (English)']['IMDb'].dropna()
films_noneng = glob_show_data[glob_show_data.category == 'Films (Non-English)']['IMDb'].dropna()

In [49]:
## performing t-test

t_statistics, p_value = stats.ttest_ind(tv_eng, tv_noneng)

In [50]:
## setting significance level = 0.05

alpha = 0.05

print(f"T-statistics value is {t_statistics} and P-Value is {p_value}")

T-statistics value is 0.4994830838794304 and P-Value is 0.6176377607433152


In [51]:
# tv.mean(), films.mean()

In [52]:
## performing ANOVA test

f_statistics, p_value_anova = stats.f_oneway(tv_eng, tv_noneng, films_eng, films_noneng)

In [53]:
## setting significance level = 0.05

alpha = 0.05

print(f"F-statistics value is {f_statistics} and P-Value is {p_value_anova}")

print("\nConclusion : \n")

if p_value_anova > alpha:
    print(f"P-Value, {p_value_anova} is greater than alpha, {alpha}, We fail to REJECT the NULL Hypothesis and conclude that there is significant difference in IMDb ratings in different categories")
else:
    print("We will REJECT the NULL Hypothesis, concludes that there is no significance differences on ratings on IMDb in different category")

F-statistics value is 8.888317567654266 and P-Value is 9.121234076879603e-06

Conclusion : 

We will REJECT the NULL Hypothesis, concludes that there is no significance differences on ratings on IMDb in different category


### 4. Is there a correlation between IMDb ratings and Rotten Tomatoes ratings for TV shows?

In [54]:
## using `pearsonr` function for calculation of correlation and p_value will be compared with significance level for testing

## filtering data
imdb = shows_data['IMDb'].dropna()
rotten_tomatoes =pd.to_numeric(shows_data['Rotten_Tomatoes'].str.strip('%'), errors='coerce').dropna()

## checking data
# print(imdb.isnull().sum(), rotten_tomatoes.isnull().sum(), imdb.shape, rotten_tomatoes.shape)

## taking equal sample size
imdb_sample = imdb.sample(1000)
rotten_tomatoes_sample = rotten_tomatoes.sample(1000)

In [55]:
# imdb_sample, 
type(rotten_tomatoes_sample[0]), rotten_tomatoes_sample[0]

(numpy.float64, 96.0)

In [56]:
## applying pearsonr function for evaluating correlation_value and p_value

from scipy.stats import pearsonr

correlation_coefficient, p_value = pearsonr(imdb_sample, rotten_tomatoes_sample)

## checking the result
print(f"The correlation coefficient is {correlation_coefficient} and P-Value between IMDb and Rotten Tomatoes is {p_value}")

The correlation coefficient is 0.06671479798738225 and P-Value between IMDb and Rotten Tomatoes is 0.034908436427596536


In [57]:
## testing results

## defining significance level

alpha = 0.05

if p_value > alpha:
    print(f"P-Value, {p_value} is learger then alpha, {alpha}, so We fail to REJECT NULL Hypothesis and Concludes that There is No Correlation between IMDb ratings and Rotten Tomatoes ratings.")
else:
    print(f"We REJECT the NULL Hypothesis and Concludes that There is a correlation, {correlation_coefficient} between IMDb ratings and Rotten Tomatoes ratings")
    

We REJECT the NULL Hypothesis and Concludes that There is a correlation, 0.06671479798738225 between IMDb ratings and Rotten Tomatoes ratings


### 5. Are there significant differences in IMDb ratings between TV shows released in different years?


In [58]:
shows_data.head(4)

,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney+,type
0,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,1.0
1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1.0
2,Money Heist,2017,18+,8.4,91%,1,0,0,0,1.0
3,Sherlock,2010,16+,9.1,78%,1,0,0,0,1.0


In [80]:
shows_data.query('Year == 2020').

(177, 10)

In [83]:
# shows_data.dropna().IMDb.value_counts()[:20].reset_index().rename(columns={'index' : 'Year', 'Year':'Values'})
# shows_data.Year.nunique(), shows_data['Year'].max() 
# shows_data['Year'].unique()

In [60]:
## filtering data
imdb_year_data = shows_data[['Year', 'IMDb']]
imdb_year_data[:10]

,Year,IMDb
0,2008,9.5
1,2016,8.8
2,2017,8.4
3,2010,9.1
4,2015,8.7
5,2005,8.9
6,2011,8.8
7,2005,8.4
8,2013,8.8
9,2005,9.2


In [87]:
imdb_year_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5610 entries, 0 to 5610
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    5610 non-null   int32  
 1   IMDb    4450 non-null   float64
dtypes: float64(1), int32(1)
memory usage: 109.6 KB


In [95]:
imdb_year_data = imdb_year_data.dropna()   # making sure no null values are there

In [96]:
imdb_year_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4450 entries, 0 to 5602
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    4450 non-null   int32  
 1   IMDb    4450 non-null   float64
dtypes: float64(1), int32(1)
memory usage: 86.9 KB


In [85]:
# ## create a function that will take the data frame and return variables which will contain `IMDb` ratings Yearwise

# # def variable_create(data):
# year_data = imdb_year_data.dropna().groupby('Year')

# datasets_by_year = {}

# for year, data in year_data:
#     datasets_by_year[year] = data['IMDb'].tolist()

#     ## create variables dynamically for each years datasets
# list_of_datasets = []
# for year, datasets in datasets_by_year.items():
#     dataset = f"dataset_{year}"   #creating variables name
#     globals()[dataset] = datasets  #storing the info
#     list_of_datasets.append(dataset)  #appending the dataset name into list
# #     return list_of_datasets
# # variable_create(imdb_year_data)


In [98]:
countries_data.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2023-03-05,Films,1,We Have a Ghost,NaN,2
1,Argentina,AR,2023-03-05,Films,2,Bad Boys for Life,NaN,2
2,Argentina,AR,2023-03-05,Films,3,The Condemned,NaN,1
3,Argentina,AR,2023-03-05,Films,4,Love at First Kiss,NaN,1
4,Argentina,AR,2023-03-05,Films,5,Tonight You're Sleeping with Me,NaN,1


In [99]:
countries_data.category.value_counts()

Films    82190
TV       82190
Name: category, dtype: int64

In [102]:
## Merge two dataframes, countries_data and shows_data based on 'Title'  

country_show_data = pd.merge(countries_data, shows_data, left_on='show_title', 
                                                        right_on='Title',
                                                        how = 'inner')


country_show_data.shape

(31664, 18)

In [103]:
country_show_data.columns

Index(['country_name', 'country_iso2', 'week', 'category', 'weekly_rank',
       'show_title', 'season_title', 'cumulative_weeks_in_top_10', 'Title',
       'Year', 'Age', 'IMDb', 'Rotten_Tomatoes', 'Netflix', 'Hulu',
       'Prime_Video', 'Disney+', 'type'],
      dtype='object')

In [107]:
country_show_data.Title.nunique(), country_show_data.show_title.nunique()

(308, 308)

In [105]:
## filtering data columns as per need
yearly_rating_data = country_show_data[['Year', 'IMDb', 'category']]

yearly_rating_data.head()

,Year,IMDb,category
0,2017,7.2,Films
1,2017,7.2,Films
2,2017,7.2,Films
3,2017,7.2,Films
4,2017,7.2,Films


In [112]:
## basic processing the data

# yearly_rating_data.Year.value_counts()

yearly_rating_data = yearly_rating_data[yearly_rating_data.category == 'TV']
yearly_rating_data.category.value_counts()

TV    29423
Name: category, dtype: int64

In [117]:
# yearly_rating_data.Year.value_counts()
yearly_rating_data = yearly_rating_data.dropna()  # droping all null values

In [118]:
yearly_rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29394 entries, 5 to 31663
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      29394 non-null  int32  
 1   IMDb      29394 non-null  float64
 2   category  29394 non-null  object 
dtypes: float64(1), int32(1), object(1)
memory usage: 803.7+ KB


In [121]:
yearly_rating_data.Year.value_counts()[:10]

2017    6081
2018    5534
2016    4857
2019    4293
2020    3671
2015    1129
2013    1012
2011     829
2014     657
1965     333
Name: Year, dtype: int64

In [125]:
## filtering data for each year

data_2017 = yearly_rating_data[yearly_rating_data.Year == 2017]['IMDb']
data_2018 = yearly_rating_data[yearly_rating_data.Year == 2018]['IMDb']
data_2016 = yearly_rating_data[yearly_rating_data.Year == 2016]['IMDb']
data_2019 = yearly_rating_data[yearly_rating_data.Year == 2019]['IMDb']
data_2020 = yearly_rating_data[yearly_rating_data.Year == 2020]['IMDb']
data_2015 = yearly_rating_data[yearly_rating_data.Year == 2015]['IMDb']
data_2013 = yearly_rating_data[yearly_rating_data.Year == 2013]['IMDb']
data_2011 = yearly_rating_data[yearly_rating_data.Year == 2011]['IMDb']
data_2014 = yearly_rating_data[yearly_rating_data.Year == 2014]['IMDb']
data_1965 = yearly_rating_data[yearly_rating_data.Year == 1965]['IMDb']

# data_2017.mean()

In [130]:
## manual testing

print([data_1965.mean(), 
      data_2011.mean(),
      data_2013.mean(),
      data_2014.mean(),
      data_2015.mean(),
      data_2016.mean(),
      data_2017.mean(),
      data_2018.mean(),
      data_2019.mean(),
      data_2020.mean()])

[7.200000000000001, 8.273703256936066, 8.446837944664033, 7.787823439878235, 8.129406554472984, 8.638768787317273, 8.072175629008388, 7.51928080954102, 7.857791754018169, 7.124707164260419]


In [136]:
## hypothesis testing

## significance level defining

alpha = 0.05

## testing within column for multiple category

f_statistics, p_value = stats.f_oneway(data_1965, 
                                       data_2020, 
                                       data_2011,
                                       data_2013,
                                       data_2014,
                                       data_2015,
                                       data_2016,
                                       data_2017,
                                       data_2018,
                                       data_2019)


## results

print(f"The statistics value of the test is {f_statistics}, and P-Value is {p_value}")


## conclusion

if p_value > alpha:
    print("The P-Value, {p_value} is greater than significance level, {alpha} and Thus we concludes that We cannot REJECT the NULL Hypothesis and that means There is no difference between IMDb ratings yearwise")
else:
    print("We REJECT the NULL Hypothesis and that concludes that there is difference between IMDb ratings yearwise.")

The statistics value of the test is 1771.2208941502229, and P-Value is 0.0
We REJECT the NULL Hypothesis and that concludes that there is difference between IMDb ratings yearwise.


### 6. Is there a relationship between the cumulative weeks in the top 10 and IMDb ratings of TV shows?


In [146]:
## merging data for tests

global_shows_data = pd.merge(global_data, shows_data, left_on = 'show_title',
                                                      right_on='Title',
                                                      how = 'inner')

global_shows_data.shape
global_shows_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 0 to 587
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        588 non-null    object 
 1   category                    588 non-null    object 
 2   weekly_rank                 588 non-null    int64  
 3   show_title                  588 non-null    object 
 4   season_title                545 non-null    object 
 5   weekly_hours_viewed         588 non-null    int64  
 6   cumulative_weeks_in_top_10  588 non-null    int64  
 7   Title                       588 non-null    object 
 8   Year                        588 non-null    int32  
 9   Age                         550 non-null    object 
 10  IMDb                        584 non-null    float64
 11  Rotten_Tomatoes             419 non-null    object 
 12  Netflix                     588 non-null    int64  
 13  Hulu                        588 non

In [158]:
# global_data.columns

## filtering data for testing
cumulative_weeks_in_top_10 = global_shows_data.query('category == ["TV (English)", "TV (Non-English)"]')['cumulative_weeks_in_top_10'].sample(300)

imdb_rating = global_shows_data.query('category == ["TV (English)", "TV (Non-English)"]')['IMDb'].sample(300)


In [159]:
# global_shows_data.IMDb
# global_shows_data.category.value_counts()
cumulative_weeks_in_top_10.shape, imdb_rating.shape

((300,), (300,))

In [162]:
## Pearson Correlation

## defining the significance level

alpha = 0.05

correlation_coefficient, p_value = stats.pearsonr(cumulative_weeks_in_top_10, imdb_rating)

## checking the result
print(f"The correlation coefficient is {correlation_coefficient} and P-Value between cumulative weeks in the top 10 and IMDb ratings of TV shows is {p_value}")

The correlation coefficient is -0.11116161895836395 and P-Value between cumulative weeks in the top 10 and IMDb ratings of TV shows is 0.054442053985616606


In [163]:
## Conclusion

if p_value > alpha:
    print(f"P-Value, {p_value} is learger then alpha, {alpha}, so We fail to REJECT NULL Hypothesis and Concludes that There is no relationship between the cumulative weeks in the top 10 and IMDb ratings of TV shows.")
else:
    print(f"We REJECT the NULL Hypothesis and Concludes that There is a correlation, {correlation_coefficient} between the cumulative weeks in the top 10 and IMDb ratings of TV shows")
    

P-Value, 0.054442053985616606 is learger then alpha, 0.05, so We fail to REJECT NULL Hypothesis and Concludes that There is no relationship between the cumulative weeks in the top 10 and IMDb ratings of TV shows.
